**Learning outcomes**

* Derive the Trotter-Suzuki decomposition.
* Use Trotterization to simulate a simple physical system.

In [4]:
import numpy as np
import pennylane as qml

Let's now consider two electrons in a magnetic field. If they are far away from each other, then the total Hamiltonian (energy) will just be a sum of Hamiltonians:
![equation](./images/H.5.1.1.png)
where $Z_0 = Z\otimes I$ is the Pauli Z acting on the first electron, and $Z_1 = I \otimes Z$ acts on the second. Since the electrons are independent, the unitary which evolves this two-electron system in time is two copies of the unitary $e^{i\alpha tZ}$, $\alpha = eB/2m_e$. In circuit form:
![equation](./images/H.5.1.2.png)

**Codercise H.5.1.**
Complete the function below for simulating two distant electrons in a magnetic field.

In [6]:
n_bits=2
dev = qml.device("default.qubit", wires=range(n_bits))

@qml.qnode(dev)
def two_distant_spins(B, time):
    """Circuit for evolving the state of two distant electrons in a magnetic field.

    Args:
        B (float): The strength of the field, assumed to point in the z direction.
        time (float): The time we evolve the electron wavefunction for.

    Returns:
        array[complex]: The quantum state after evolution.
    """
    e = 1.6e-19
    m_e = 9.1e-31
    alpha = B*e/(2*m_e)

    qml.RZ(phi=-2 * alpha * time, wires=0)
    qml.RZ(phi=-2 * alpha * time, wires=1)

    return qml.state()

Another way to express this circuit is in matrix form,$U(t) = e^{i\alpha tZ_0}e^{i\alpha tZ_1}$ . Schrödinger's equation tells us, on the other hand, that $U(t) = e^{-i\hat{H}/\hbar} = e^{i \alpha t(Z_0 Z_1)}$. These two expressions appear to be equal by virtue of the rule $e^{x+y} = e^xe^y$. But not so fast! We are dealing with matrices rather than numbers, and this makes a huge change.
It turns out that, for matrices $A$ and $B$,$e^{A+B} = e^Ae^B$ is only tru when $A$ and $B$ can be freely reordered (commute), or $AB = BA$. We say such metrices **commute**. When term do not commute, exponentiating their sum is hard. As a concrete example, consider two nearby electrons, who can feel each other's magnetic field. The Hamiltonian will now include an interaction term since the spins of these electrons would like to anti-align:
![picture](./images/H.5.2.1.png)
For simplicity, we'll make this interaction term proportional to $X_0X_1$:
![equation](./images/H.5.2.2.png)
where $J$ depends on how close they are. (The factor of $(\hbar / 2)^2$) comes from converting spins into Pauli operators). To time evolve this system, we need to exponentiate these non-commuting terms, which is hard.

Thankfully, there is a beautiful approximation called the **Trotter-Suzuki decomposition**:
$$e^{A+B} = \lim_{n\to\infty}(e^{A/n}e^{B/n})^n$$
This lead to a simple algorithm called **Trotterization** for simulating a quantum system.
Suppose we have a Hamiltonian $\hat{H} = \hat{H_1} + \hat{H_2}$, where $\hat{H_1}$ and $\hat{H_2}$ can be easily exponentiated in our computer, but don't commute. Then Trotter-Suzuki tells us that, for some large number of steps $n$:
$$U(t) = e^{-it\hat{H}/\hbar} = e^{(-it\hat{H_1}/\hbar) + (-it\hat{H_2}/\hbar)} \approx [e^{-i(t/n)\hat{H_1}/\hbar} e^{-i(t/n)\hat{H_2}/\hbar}] = [U_1(t/n)U_2(t/n)]^n$$
, where $U_1(t)$ and $U_2(t)$ are the unitaries associated with exponentiating the terms $\hat{H_1}$ and $\hat{H_2}$. Thus, we can approximate $U(t)$ with the circuit:
![circuit](./images/H.5.2.3.png)
for some large $n$.

**Codercise H.5.2.**
Write a circuit to Trotterize the evolution of two electrons with Hamiltonian:
$$\hat{H} = -\frac{\hbar Be}{2m_e}(Z_0 +Z_1)+\frac{J\hbar^2}{4}X_0X_1$$
We've defined constants $\alpha = Be/2m_e$ and $\beta = -J\hbar/4$, so that
$$-\frac{\hat{H}}{\hbar} = \alpha(Z_0 +Z_1)+ \beta X_0X_1$$.
*Tip* For exponentiating $X_0X_1$, you will find [qml.PauliRot](https://docs.pennylane.ai/en/stable/code/api/pennylane.PauliRot.html) helpful.

Solution(calculations are included in the notes):
$U(t) = e^{-it\hat{H}/\hbar} = e^{i\alpha t(Z_0 + Z_1) +i \beta t X_0X_1} = [e^{i \alpha t(Z_0 + Z_1)/n}e^{i \beta t X_0X_1}]^n = [e^{i \alpha t Z_0/n}e^{i \alpha t Z_1/n}e^{i \beta t X_0X_1/n}]^n$

In [8]:
@qml.qnode(dev)
def two_close_spins_X(B, J, time, n):
    """Circuit for evolving the state of two electrons with an X coupling.

    Args:
        B (float): The strength of the field, assumed to point in the z direction.
        J (float): The strength of the coupling between electrons.
        time (float): The time we evolve the electron wavefunction for.
        n (int): The number of steps in our Trotterization.

    Returns:
        array[complex]: The quantum state after evolution.
    """
    e = 1.6e-19
    m_e = 9.1e-31
    alpha = B*e/(2*m_e)
    hbar = 1e-34
    beta = -J*hbar/4

    # [exp(i*alfa*t*Z_0\n)exp(i*alfa*t*Z_1\n)exp(i*beta*t*X_0*X_1\n)]^n
    for _ in range(n):
        # PauliRot(phi, P) = exp(-i phi/2 P)

        # exp(i*beta*t*X_0*X_1\n)
        qml.PauliRot(-2*beta * time/n,'XX',wires=[0,1])
        # exp(i*alfa*t*Z_1\n)
        qml.PauliRot(-2*alpha*time/n,'Z',wires=1)
        # exp(i*alfa*t*Z_0\n)
        qml.PauliRot(-2*alpha*time/n,'Z',wires=0)

    return qml.state()

A more realistic Hamiltonian for two nearby electrons involves interactions proportional to $Y_0Y_1$ and $Z_0Z_1$ in addition to $X_0X_1$:
$$\hat{H} = -\frac{\hbar Be}{2m_e}(Z_0+Z_1)+\frac{\hbar^2}{4}(J_xX_0X_1+J_YY_0Y_1+J_ZZ_0Z_1).$$
For case like this, we can use a generalization of the Trotter-Suzuki formula for a sum of $L$ terms:
$$e^{A_1+A_2+...+A_L} = \lim_{n\to\infty}(e^{A_1/n}e^{A_2/n}...e^{A_L/n})^n$$
Thus, for a Hamiltonian $\hat{H} = \hat{H_1} + \hat{H_2} + ... + \hat{H_L}$, we can replace the unitary $U(t)$ with a circuit:
![circuit](./images/H.5.2.4.png)
Although we can code this time evolution by hand, it quickly becomes tedious. Thankfully, in Pennylane we can input the Hamiltonian and automatically Trotterize.

Solution(calculations are included in the notes):
$\frac{\hat{H}}{\hbar} = -\alpha(Z_0 +Z_1) + \frac{\hbar}{4}(J_xX_0X_1 +J_yY_0Y_1+J_zZ_0Z_1)$

In [10]:
def ham_close_spins(B, J):
    """Creates the Hamiltonian for two close spins.

    Args:
        B (float): The strength of the field, assumed to point in the z direction.
        J (list[float]): A vector of couplings [J_X, J_Y, J_Z].

    Returns:
        qml.Hamiltonian: The Hamiltonian of the system.
    """
    e = 1.6e-19
    m_e = 9.1e-31
    alpha = B*e/(2*m_e)
    hbar = 1e-34

    beta = hbar/4
    coeffs = [beta*J[2],beta*J[1],beta*J[0],-alpha,-alpha]
    obs = [qml.PauliZ(wires=0)@qml.PauliZ(wires=1),qml.PauliY(wires=0)@qml.PauliY(wires=1), qml.PauliX(wires=0)@qml.PauliX(wires=1),qml.PauliZ(wires=1),qml.PauliZ(wires=0)]

    return qml.Hamiltonian(coeffs, obs)
print(ham_close_spins(5, [1,2,3]))

  (-439560439560.4396) [Z1]
+ (-439560439560.4396) [Z0]
+ (2.5e-35) [X0 X1]
+ (5e-35) [Y0 Y1]
+ (7.5e-35) [Z0 Z1]


To Trotterize, we can use [qml.ApproxTimeEvolution](https://docs.pennylane.ai/en/stable/code/api/pennylane.ApproxTimeEvolution.html), which simply takes a Hamiltonian, a time to evolve, and a number of steps for the Trotterization.

**Codercise H.5.4.** Use the function `ham_close_spins(B, J)` from the previous exercise, along with the `qml.ApproxTimeEvolution` method, to simulate evolution under couplings $J = (J_x,J_y,J_z)$ and a magnetic field of strength B in the $z$-direction.
Hint:
we calculate $\frac{\hat{H}}{\hbar}$, because `qml.ApproxTimeEvolution` utility the general time-evolution operator for a time-independent Hamiltonian given by:
$$U(t) = e^{-iHt}$$

In [ ]:
@qml.qnode(dev)
def two_close_spins(B, J, time, n):
    """Circuit for evolving the state of two nearby electrons with an arbitrary coupling.

    Args:
        B (float): The strength of the field, assumed to point in the z direction.
        J (array[float]): The coupling strengths J = [J_X, J_Y, J_Z] between electrons.
        time (float): The time we evolve the electron wavefunction for.
        n (int): The number of steps in our Trotterization.

    Returns:
        array[complex]: The quantum state after evolution.
    """
    hamiltonian = ham_close_spins(B, J)
    qml.ApproxTimeEvolution(hamiltonian,time,n)

    return qml.state()
